# Data Import & Preprocessing 
### Project: Dog Breed Image Classifier 
Team Members: Taylor Witte, Donald Yu, Praveen Manimaran, Vitush Agarwal, Parker Aman
UCSD Spring 2024 232R Big Data Analytics Using Spark 


## Enviroment Setup

In [ ]:
## Dataset
path = "/kaggle/input/143-different-dog-breeds-image-classifier/pet classifer image directory"

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
# Import necessary modules
from pyspark.sql import SparkSession

# Create a SparkSession object
spark = SparkSession.builder \
    .appName("BigDataGroupProject") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
# Define the path to the CSV file
file_path = "/Users/praveenmanimaran/Desktop/usa_00005.csv" #change this to your own!!

# Read the CSV file into a DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the schema of the DataFrame
df.printSchema()

# Show the first few rows of the DataFrame
df.show(5)